# Imports

In [1]:
from utils import *
SEED = 0

# Parámetros

In [2]:
# FPCA
N_BASIS = 15
N_COMPONENTS = 3

# WGAN
N_EPOCHS = 2000
N_CRITIC = 5
LAMBDA_GP = 0.5
BATCH_SIZE = 128
LATENT_DIM = 1
CONDITION_DIM = 1
INPUT_DIM = 1
G_HIDDEN_LAYERS = 4
G_HIDDEN_DIM = 16
G_LEAKY_RELU = 0.1
D_HIDDEN_LAYERS = 2
D_HIDDEN_DIM = 64
D_DROPOUT = 0.4
D_LEAKY_RELU = 0.1
LR_GEN = 0.0002
LR_DIS = 0.0002
B1 = 0.5
B2 = 0.999

# Datos

## Carga y Análisis

In [ ]:
data = pd.read_excel( 'mltan_database_2022.xlsx' )

data.rename(columns = {'norad':'NORAD'}, inplace=True)

print(f'{(len(data))} observaciones')
print(f'Primera observación: {data.epoch.min()}')
print(f'Última observación: {data.epoch.max()}')

In [ ]:
# En primer lugar filtramos observaciones para las cuales no se tiene el dato de masa

print('No contamos con la masa para un {:.2f}% de las observaciones'.format( np.mean(data.launch_mass.isna())*100 ))
data = data[data.launch_mass.notna()].reset_index(drop=True)
print(f'{len(data)} observaciones')

In [ ]:
# Acotamos la muestra a un subgrupo de satélites
# La principal distinción es por el tipo de órbita, siendo las principales LEO (órbita baja) y GEO (órbita geoestacionaria - muy alta)
# LEO es la fracción más grande

data.groupby('orbit_class')['cospar_norad'].count() / data.shape[0]

In [ ]:
# Distintas masas se observan en los 4 tipos de órbitas. En general, en LEO se encuentran satélites por debajo de 1.000 kgs
# GEO presenta una distribución totalmente distinta con muchos satélites pesando varias toneladas
# Al ser la categoría con más observaciones, decidimos quedarnos sólo con los satélites de LEO

plt.figure(dpi=100, figsize=(10,8))
classes = pd.unique(data.orbit_class)

for i, orbit_class in zip( range(len(classes)), classes ):

  plt.subplot(2,2,i+1)
  sns.histplot(data[data.orbit_class == orbit_class].launch_mass, kde=True, bins=30)
  plt.xlabel('launch_mass [kgs]')
  plt.title(orbit_class)

plt.tight_layout()
plt.show()

In [ ]:
data = data[data.orbit_class == 'LEO'].reset_index(drop=True)

In [ ]:
# Un único dato estaba clasificado como Earth Observation bajo la columna Users (mal clasificado)
# Se trata del BRIK-II, lanzado por la fuerza aérea de los Países Bajos. Editamos este dato:

data.loc[data.users == 'Earth Observation', 'users'] = 'Military'

In [ ]:
# Análisis según tipo de usuario

data.groupby('users')['cospar_norad'].count()

In [ ]:
plt.figure(dpi=100, figsize=(10,8))
users = pd.unique(data.users)

for i, user in zip( range(len(users)), users ):

  plt.subplot(2,2,i+1)
  sns.histplot( data[data.users == user].launch_mass, bins=30, kde=True )
  plt.title(user)

plt.tight_layout()
plt.show()

In [ ]:
# Si bien LEO se caracteriza por satélites pequeños, vemos algunos datos pesando varias toneladas
# Viéndolos en detalle vemos que por encima de los 4.000 kgs, todos los satélites pertenecen a usuarios
# militares o gubernamentales

# optamos por descartar todas las observaciones por encima de los 4.000 kilos

print('Un {:.3f}% de los satélites en LEO superan los 4.000 kilos'.format(np.mean(data.launch_mass >= 4000)))
data.sort_values(by='launch_mass', ascending=False).head(40)

In [ ]:
data = data[data.launch_mass < 4000].reset_index(drop=True)

## Agrupación por año

Habiéndonos quedado con los satélites lanzados a LEO por debajo de los 4.000 kilos, pasamos a agrupar por año

In [ ]:
data = data[['launch_mass', 'year']].sort_values(by='year').reset_index(drop=True) # Me quedo con masa y año de lanzamiento
print(data.groupby('year').launch_mass.count())

# Como buscamos ahora ajustar una distribución para cada año, descartamos los años con pocas observaciones
# Arbitrariamente descartamos los años con menos de 10 lanzamientos
# Notar que la caída entre 2021 y 2022 se debe a que todavía no terminó el año al momento de hacer este análisis

data = data[data.year >= 1998].reset_index(drop=True)

In [ ]:
# Se observa una gran conentración de las masas en torno a los 200 kgs para los últimos años
# A juzgar por los picos de las densidades, para los primeros años esta concentración no era tan alta, siendo más comunes satélites más pesados
# Vale aclarar que la gran concentración de los últimos años se debe a la constelación Starlink de SpaceX
# En una muestra de 5600 observaciones, casi la mitad pertenece a esta constelación formada por satélites idénticos de entre 230 y 260 kilos, lanzados entre 2019 y 2022

data_plot = data.copy()

bandwidths = 1.06 * data_plot.groupby('year').launch_mass.std() * ( data_plot.groupby('year').launch_mass.count() ** (-0.2) )

kdes = dict()

# Ajusto con KDE
for t in data_plot.year.unique():
  kde = KernelDensity(
      kernel='gaussian',
      bandwidth=bandwidths[t]).fit( data_plot.loc[data_plot.year==t, ['launch_mass']].values )
  kdes[t] = kde

kdes = get_kdes(data_plot, 'launch_mass', 'year', data_plot.year.unique())

# Evalúo las log-densidades sobre una grilla
grid_points = list(np.linspace(min(data_plot.launch_mass)-4, max(data_plot.launch_mass)+4, 1000))
data_matrix = [[kdes[t].score( np.array([[point]]) ) for point in grid_points] for t in data_plot.year.unique()]

plt.figure(dpi=100, figsize=(8,8))
for i, d in zip(np.linspace(0.3,0.7, len(data_matrix)), data_matrix):
  plt.plot(grid_points, np.exp(d), color=blue(i))

plt.gca().set_ylim(bottom=0)
plt.ylabel('Density')
plt.xlabel('Kgs')
plt.show()

In [ ]:
# COMENTARIO

# Una boludez... calculo todo de nuevo solo para no graficar 2022

# data_plot = data[data.year < 2020].copy()


# bandwidths = 1.06 * data_plot.groupby('year').launch_mass.std() * ( data_plot.groupby('year').launch_mass.count() ** (-0.2) )

# kdes = dict()

# # Ajusto con KDE
# for t in data_plot.year.unique():
#   kde = KernelDensity(
#       kernel='gaussian',
#       bandwidth=bandwidths[t]).fit( data_plot.loc[data_plot.year==t, ['launch_mass']].values )
#   kdes[t] = kde

# # Evalúo las log-densidades sobre una grilla
# grid_points = list(np.linspace(min(data_plot.launch_mass)-4, max(data_plot.launch_mass)+4, 1000))
# data_matrix = [[kdes[t].score( np.array([[point]]) ) for point in grid_points] for t in data_plot.year.unique()]


# plt.figure(dpi=100, figsize=(8,8))
# for i, d in zip(np.linspace(0.3,0.7, len(data_matrix)), data_matrix):
#   plt.plot(grid_points, np.exp(d), color=blue(i))

# plt.gca().set_ylim(bottom=0)
# plt.ylabel('Density')
# plt.xlabel('Kgs')
# plt.show()

# Proyecciones

Ahora aplicamos los 3 métodos que desarrollamos para predecir la evolución de las densidades

Ajustaremos los 3 modelos hasta el 2021, y compararemos las predicciones para 2022 con la distribución observada para dicho año a la fecha

In [ ]:
data = data.rename(columns={'year':'time'})
vars = ['launch_mass']
X_train = data[data.time < 2022].reset_index(drop=True)
X_test = data[data.time == 2022].reset_index(drop=True)
H_STEPS = 1
time_periods = pd.unique(X_train.time)
forecast_periods = np.arange(time_periods[-1]+1, time_periods[-1]+1+H_STEPS)

# bandwidths para la estimación con KDE
bandwidths = 1.06 * data.groupby('time').launch_mass.std() * ( data.groupby('time').launch_mass.count() ** (-0.2) )

kdes = get_kdes(data, 'launch_mass', 'time', pd.unique(data.time))
    
# Evalúo las log-densidades sobre una grilla
grid_points = np.linspace(min(data.launch_mass)-4, max(data.launch_mass)+4, 1000)
data_matrix = [kdes[t].score_samples(grid_points.reshape(-1,1)) for t in time_periods]

In [ ]:
plt.figure(dpi=100, figsize=(10,8))

ax1 = plt.subplot(2,2,1)
ax2 = ax1.twinx()
ax1.plot(grid_points, np.exp(data_matrix[0]), 'darkorange')
ax2.hist(data[data.time==time_periods[0]].launch_mass, bins=30, alpha=0.8)
plt.xlabel('launch_mass [kgs]')
plt.title(time_periods[0])

ax1 = plt.subplot(2,2,2)
ax2 = ax1.twinx()
ax1.plot(grid_points, np.exp(data_matrix[5]), 'darkorange')
ax2.hist(data[data.time==time_periods[5]].launch_mass, bins=30, alpha=0.8)
plt.xlabel('launch_mass [kgs]')
plt.title(time_periods[5])

ax1 = plt.subplot(2,2,3)
ax2 = ax1.twinx()
ax1.plot(grid_points, np.exp(data_matrix[15]), 'darkorange')
ax2.hist(data[data.time==time_periods[15]].launch_mass, bins=30, alpha=0.8)
plt.xlabel('launch_mass [kgs]')
plt.title(time_periods[15])

ax1 = plt.subplot(2,2,4)
ax2 = ax1.twinx()
ax1.plot(grid_points, np.exp(data_matrix[-1]), 'darkorange')
ax2.hist(data[data.time==time_periods[-1]].launch_mass, bins=30, alpha=0.8)
plt.xlabel('launch_mass [kgs]')
plt.title(time_periods[-1])

plt.tight_layout()

plt.show()

## Paramétrico

Ajustamos mixturas de dos log normales

In [ ]:
np.random.seed(SEED)
mixtures = Mixtures(X_train, vars, n_components=2)
mixtures.fit()
mixtures.forecast(steps=H_STEPS, predict=['mean','weight'])

In [ ]:
# COMENTARIO: Usar get_kl

output_parametrico = np.exp(mixtures.new_mixtures[0].log_probability(grid_points))
KL_parametrico = sum(rel_entr(output_parametrico[1:-1], np.exp(data_matrix[-1])[1:-1]))

plt.figure(dpi=100, figsize=(10,8))

plt.plot(grid_points, output_parametrico, 'darkorange', label='Estimado')
plt.plot(grid_points, np.exp(data_matrix[-1]), 'steelblue', label='Target')
plt.legend()
plt.gca().set_ylim(bottom=0)
plt.gca().set_xlim(left=0)
plt.title('2022 - Paramétrico - KL {:.3f}'.format(KL_parametrico))

plt.xlabel('launch_mass')
plt.ylabel('density')

plt.show()

In [ ]:
# Estimo los parámetros para el grupo de test

mix_test = Mixtures(X_test, vars, n_components=2)
mix_test.fit()

In [ ]:
gs = gridspec.GridSpec(2, 2)

plt.figure(dpi=100, figsize=(10,10))

ax = plt.subplot(gs[0, :]) # row 0, col 0

plt.xlabel('Año')
plt.title('Evolución de las medias')
plt.plot(mixtures.medias)
plt.plot(mixtures.pred_medias.m_0, 'steelblue', marker='x', label='Media 1 - Predicción')
plt.plot(mixtures.pred_medias.m_1, 'darkorange', marker='x', label='Media 2 - Predicción')

plt.plot(mix_test.medias.m_0, 'steelblue', marker='o', label='Media 1')
plt.plot(mix_test.medias.m_1, 'darkorange', marker='o', label='Media 2')
plt.tight_layout()
plt.legend()


ax = plt.subplot(gs[1, 0]) # row 0, col 1


plt.xlabel('Año')
plt.title('Evolución de la ponderación')
plt.plot(mixtures.weights.w_0)

plt.scatter(forecast_periods, mixtures.new_weights, marker='x', label='Ponderación 1 - Predicción')
plt.plot(mix_test.weights.w_0, 'steelblue', marker='o', label='Ponderación 1')
plt.tight_layout()
plt.legend()


ax = plt.subplot(gs[1, 1]) # row 1, span all column

plt.xlabel('Año')
plt.title('Evolución de los desvíos')
plt.plot(mixtures.desvios)
plt.scatter(forecast_periods, mixtures.fixed_desvios[0], marker='x', label='Desvío 1 - Predicción')
plt.scatter(forecast_periods, mixtures.fixed_desvios[1], marker='x', label='Desvío 2 - Predicción')
plt.plot(mix_test.desvios.d_0, 'steelblue', marker='o', label='Desvío 1')
plt.plot(mix_test.desvios.d_1, 'darkorange', marker='o', label='Desvío 2')
plt.tight_layout()
plt.legend()

plt.show()


## FPCA

In [ ]:
fpca_sats = Fpca(data_matrix, grid_points, n_basis=N_BASIS)

plt.figure(figsize=(8, 6), dpi=80)

for n, i in zip(np.linspace(0.1,0.9,len(time_periods)), range(len(time_periods))):
  plt.plot(grid_points, np.exp(fd_basis[i](grid_points).reshape(-1)), color=viridis(n))

plt.title('Representación en BSplines')
plt.xlim((grid_points[0], grid_points[-1]))
plt.gca().set_ylim(bottom=0)
plt.show()


for n, i in zip(np.linspace(0.1,0.9,T), TIME_PERIODS):
  plt.plot(grid_points, np.exp(fpca_switch.fd[i](grid_points).reshape(-1)), color=VIRIDIS(n))
plt.gca().set_ylim(bottom=0)
plt.show()